In [53]:
import sklearn

In [54]:
import pandas as pd

In [55]:
data = pd.read_csv('../input/cleaned_data.csv')
data = data.dropna()

In [56]:
y = data['Happiness Score']

In [57]:
X = data.iloc[:,3:]

In [61]:
from sklearn import preprocessing
from scipy import stats 
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats



X_normed = preprocessing.normalize(X)
X_normed = sm.add_constant(X_normed)

est = sm.OLS(y, X_normed)
est2 = est.fit()
print(data.columns[3:])
print(est2.summary())

# reg = LinearRegression().fit(X_normed, y)
# for idx, name in zip(range(15),data.columns[3:]):
#     print(name, ":", reg.coef_[idx])
# print("intercept:",reg.intercept_)



Index(['Log GDP per capita', 'Social support',
       'Healthy life expectancy at birth', 'Freedom to make life choices',
       'Generosity', 'Perceptions of corruption', 'Positive affect',
       'Negative affect', 'Confidence in national government',
       'Democratic Quality', 'Delivery Quality',
       'Standard deviation of happiness score by country-year',
       'Standard deviation/Mean of happiness score by country-year',
       'GINI index (World Bank estimate), average 2000-15',
       'gini of household income reported in Gallup, by wp5-year'],
      dtype='object')
                            OLS Regression Results                            
Dep. Variable:        Happiness Score   R-squared:                       0.910
Model:                            OLS   Adj. R-squared:                  0.909
Method:                 Least Squares   F-statistic:                     901.8
Date:                Wed, 05 Dec 2018   Prob (F-statistic):               0.00
Time:              

In [47]:
len(data.columns)

18